In [ ]:
%matplotlib notebook
from itertools import chain
from math import atan2, cos, pi, sin
import ipywidgets as widgets
from matplotlib.figure import Figure
from matplotlib.ticker import FixedLocator, FixedFormatter
import odeanimate.plots
from odeanimate.curve import Curve1D
from odeanimate.vector import Vector2D
from odeanimate.domains import Interval
from odeanimate.jupyter import display_return

In [ ]:
fig = Figure()
ax = fig.add_subplot(projection="odeanimate")

In [ ]:
def phase_calculation(w, A, B):
    t_1 = atan2(A, B) / w

    @Curve1D
    def x_sin(t):
        return A * sin(w * t)

    @Curve1D
    def x_cos(t):
        return B * cos(w * t)

    x = x_sin + x_cos

    amp = x(t_1)
    if amp < 0:
        phase = t_1 + pi / w
        amp = abs(amp)
    else:
        phase = t_1
    phase = -phase
    return widgets.VBox(
        [
            widgets.HTMLMath(f"$x(t) = V\cos(\omega t + \phi_0)$"),
            widgets.HTMLMath(f"$V \\approx {amp}$"),
            widgets.HTMLMath(f"$\\phi_0 \\approx {phase}$"),
        ]
    )


def variating_amplitudes(w, A, B):
    @Curve1D
    def x_sin(t):
        return A * sin(w * t)

    @Curve1D
    def x_cos(t):
        return B * cos(w * t)

    x = x_sin + x_cos

    T = 2 * pi / w

    amp = max(abs(A), abs(B)) * 1.43
    interval = Interval(0, T)

    t_cut = atan2(B, A) / w
    t_destruct = atan2(-B, A) / w
    a_half = x_sin(t_cut)
    x_guides = sorted(
        [
            *[i * T / 4 for i in range(5)],
            *[t_cut + i * T / 2 for i in range(3)],
            *[t_destruct + i * T / 2 for i in range(3)],
        ]
    )

    cuts = [
        v
        for v in chain(
            *[
                [
                    Vector2D(t_cut + i * T / 2, ((-1) ** (i)) * a_half),
                    Vector2D(t_destruct + i * T / 2, ((-1) ** (1 + i)) * a_half),
                    Vector2D(t_destruct + i * T / 2, ((-1) ** (i)) * a_half),
                ]
                for i in range(4)
            ]
        )
        if 0 <= v.x <= T
    ]

    period_markers_minor = [v.x for v in cuts]
    period_markers = [
        (T / 4, r"$\frac{T}{4}$"),
        (T / 2, r"$\frac{T}{2}$"),
        (3 * T / 4, r"$\frac{3T}{4}$"),
        (T, "$T$"),
    ]

    amp_markers = sorted(
        [(abs(A), "$A$"), (abs(B), "$B$"), (-abs(A), "$-A$"), (-abs(B), "$-B$")],
        key=lambda i: i[0],
        reverse=True,
    )

    amp_markers_minor = sorted(
        [
            (abs(a_half), r"$\frac{a}{2}$"),
            (abs(a_half * 2), "$a$"),
            (-abs(a_half), r"$-\frac{a}{2}$"),
            (-abs(a_half * 2), "$-a$"),
        ],
        key=lambda i: i[0],
        reverse=True,
    )

    fig = Figure(figsize=(12, 6))
    gs = fig.add_gridspec(2, 1)

    ax_singles = fig.add_subplot(gs[0, 0], projection="odeanimate")
    ax_sum = fig.add_subplot(gs[1, 0], projection="odeanimate")

    for ax in [ax_singles, ax_sum]:
        ax.set_limits(
            Interval(0, 1.05 * T), Interval(*sorted([-amp * 1.05, amp * 1.05]))
        )
        ax.tick_params("x", length=5)
        ax.set_markers(
            x_markers=period_markers,
            y_markers=amp_markers,
        )
        ax.set_markers(
            x_markers=period_markers_minor, y_markers=amp_markers_minor, major=False
        )
        ax.grid(which="major", color="#CCCCCC", linestyle="--")
        ax.grid(which="minor", color="#CCCCCC", linestyle=":")

    ax_singles.add(x_sin, interval=interval, delta=T / 50, color="red")
    ax_singles.add(x_cos, interval=interval, delta=T / 50, color="blue")
    ax_singles.add(cuts, point=True, color="green")

    ax_sum.add(x, interval=interval, delta=T / 50, color="purple")
    ax_sum.add([Vector2D(t, x(t)) for t in x_guides], point=True, color="green")

    return fig  # , (ax_singles, ax_sum)


variating_amplitudes(0.1, 1, 3)

In [ ]:
amplitud_limit = 10

widget_kwargs = dict(
    value=1,
    min=-amplitud_limit,
    max=amplitud_limit,
    step=0.05,
)

_A = widgets.FloatSlider(
    description=r"$A$",
    **widget_kwargs,
)
_B = widgets.FloatSlider(
    description=r"$B$",
    **widget_kwargs,
)
_w = widgets.FloatSlider(
    description=r"$\omega$",
    value=1,
    min=0.01,
    max=10,
    step=0.005,
)
_t = widgets.FloatSlider(
    description=r"$t$",
    value=10,
    min=0.01,
    max=10,
    step=0.005,
)
out_numbers = widgets.interactive_output(
    display_return(phase_calculation), {"w": _w, "A": _A, "B": _B}
)
out = widgets.interactive_output(
    display_return(variating_amplitudes), {"w": _w, "A": _A, "B": _B}
)
None

In [ ]:
widgets.VBox(
    [
        widgets.HBox(
            [
                widgets.VBox(
                    [
                        _w,
                        _A,
                        _B,
                    ]
                ),
                widgets.VBox(
                    [
                        widgets.HTMLMath(
                            r"$x(t) = A\sin(\omega t) + B \cos(\omega t)$"
                        ),
                        widgets.HTMLMath(r"$T = \frac{2\pi}{\omega}$"),
                        widgets.HTMLMath(
                            r"$\frac{a}{2} = \frac{\arctan(\frac{B}{A})}{\omega}$"
                        ),
                    ]
                ),
                out_numbers,
            ]
        ),
        out,
    ]
)